# Introduction to Keras for Engineers

https://keras.io/getting_started/intro_to_keras_for_engineers/

In [1]:
import numpy as np
import tensorflow as tf
import keras

## Data Preprocessing with Keras

### Example: turning strings into sequences of integer word indices

In [2]:
from tensorflow.keras.layers import TextVectorization

training_data = np.array([["This is the 1st sample."], ["And here's the 2nd sample."]])

vectorizer = TextVectorization(output_mode="int")

vectorizer.adapt(training_data)

integer_data = vectorizer(training_data)
print(integer_data)

tf.Tensor(
[[4 5 2 9 3]
 [7 6 2 8 3]], shape=(2, 5), dtype=int64)


### Example turning strings into sequence of one-hot encoded bigrams

In [3]:
from tensorflow.keras.layers import TextVectorization

training_data = np.array([["This is the 1st sample."], ["And here's the 2nd sample."]])

vectorizer = TextVectorization(output_mode="binary", ngrams=2)

vectorizer.adapt(training_data)

integer_data = vectorizer(training_data)
print(integer_data)


tf.Tensor(
[[0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]], shape=(2, 17), dtype=float32)


### Example: normalizing features

In [4]:
from tensorflow.keras.layers import Normalization

training_data = np.random.randint(0, 256, size=(64, 200, 200, 3)).astype("float32")

normalizer = Normalization(axis=-1)
normalizer.adapt(training_data)

normalized_data = normalizer(training_data)
print("var: %.4f" % np.var(normalized_data))
print("mean: %.4f" % np.mean(normalized_data))


var: 1.0005
mean: 0.0000


### Example: rescaling & center-cropping images

In [5]:
from tensorflow.keras.layers import CenterCrop
from tensorflow.keras.layers import Rescaling

training_data = np.random.randint(0, 256, size=(64, 200, 200, 3)).astype("float32")

cropper = CenterCrop(height=150, width=150)
scaler = Rescaling(scale=1.0 / 255)

output_data = scaler(cropper(training_data))
print("shape:", output_data.shape)
print("min:", np.min(output_data))
print("max:", np.max(output_data))

shape: (64, 150, 150, 3)
min: 0.0
max: 1.0


## Building models with the Keras Functional API

In [6]:
dense = keras.layers.Dense(units=16)

In [7]:
inputs = keras.Input(shape=(None, None, 3))

In [8]:
from tensorflow.keras import layers

x = CenterCrop(height=150, width=150)(inputs)

x = Rescaling(scale=1.0 / 255)(x)

x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)

x = layers.GlobalAveragePooling2D()(x)

num_classes = 10
outputs = layers.Dense(num_classes, activation="softmax")(x)

In [9]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [10]:
data = np.random.randint(0, 256, size=(64, 200, 200, 3)).astype("float32")
processed_data = model(data)
print(processed_data.shape)

(64, 10)


In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 center_crop_1 (CenterCrop)  (None, 150, 150, 3)       0         
                                                                 
 rescaling_1 (Rescaling)     (None, 150, 150, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        9248      
                                                             

## Training models with fit()

In [12]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.CategoricalCrossentropy())

In [13]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [14]:
#model.fit(numpy_array_of_samples, numpy_array_of_labels,
#          batch_size=32, epochs=10)

In [15]:
# Get the data as Numpy arrays
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Build a simple model
inputs = keras.Input(shape=(28, 28))
x = layers.Rescaling(1.0 / 255)(inputs)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

# Train the model for 1 epoch from Numpy data
batch_size = 64
print("Fit on NumPy data")
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=1)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 rescaling_2 (Rescaling)     (None, 28, 28)            0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               100480    
                                                                 
 dense_3 (Dense)             (None, 128)               16512     
                                                                 
 dense_4 (Dense)             (None, 10)                1290      
                                                                 
Total params: 118282 (462.04 KB)
Trainable params: 118282 (

In [16]:
print(history.history)

{'loss': [0.2691708207130432]}


### Keeping track of performance metrics

#### Monitoring Metrics

In [17]:
#dataset = 
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)
history = model.fit(x_train, y_train, epochs=1)

1875/1875 [==============================] - 5s 2ms/step - loss: 0.1190 - acc: 0.9641


#### Passing validation data to fit()

In [18]:
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

history = model.fit(x_train, y_train, epochs=1, validation_data=val_dataset)

1875/1875 [==============================] - 5s 2ms/step - loss: 0.0757 - acc: 0.9760 - val_loss: 0.0796 - val_acc: 0.9752


#### Using callbacks for checkpointing (and more)

In [19]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='./models/model_{epoch}',
        save_freq='epoch')
]
model.fit(x_train, y_train, epochs=2, callbacks=callbacks)

Epoch 1/2
1855/1875 [============================>.] - ETA: 0s - loss: 0.0585 - acc: 0.9812INFO:tensorflow:Assets written to: ./models\model_1\assets


INFO:tensorflow:Assets written to: ./models\model_1\assets


1875/1875 [==============================] - 5s 3ms/step - loss: 0.0586 - acc: 0.9812
Epoch 2/2
1868/1875 [============================>.] - ETA: 0s - loss: 0.0449 - acc: 0.9852INFO:tensorflow:Assets written to: ./models\model_2\assets


INFO:tensorflow:Assets written to: ./models\model_2\assets


1875/1875 [==============================] - 5s 3ms/step - loss: 0.0450 - acc: 0.9851


#### Monitoring training progress with TensorBoard

In [20]:
callbacks = [
    keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(x_train, y_train, epochs=2, callbacks=callbacks)

Epoch 1/2
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0361 - acc: 0.9876
Epoch 2/2
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0299 - acc: 0.9904


#### After fit(): evaluating test performance & generating predictions on new data

In [21]:
loss, acc = model.evaluate(val_dataset)  # returns loss and metrics
print("loss: %.2f" % loss)
print("acc: %.2f" % acc)

157/157 [==============================] - 0s 2ms/step - loss: 0.0885 - acc: 0.9732
loss: 0.09
acc: 0.97


##### generate NumPy arrays of predictions

In [22]:
predictions = model.predict(val_dataset)
print(predictions.shape)

157/157 [==============================] - 0s 2ms/step
(10000, 10)


## Finding the best model configuration with hyperparameter tuning

In [23]:
def build_model(hp):
    inputs = keras.Input(shape=(784,))
    x = layers.Dense(
        units=hp.Int('units', min_value=32, max_value=512, step=32),
        activation='relu')(inputs)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [24]:
import keras_tuner

tuner = keras_tuner.tuners.Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=100,
    #max_trials=200,
    executions_per_trial=2,
    directory='./tuners')

Using TensorFlow backend
Reloading Tuner from ./tuners\untitled_project\tuner0.json


In [25]:
#tuner.search(inputs, outputs, validation_data=val_dataset)

In [26]:
#models = tuner.get_best_models(num_models=2)

In [27]:
#tuner.results_summary()